<a href="https://colab.research.google.com/github/Fulim13/Storyteller/blob/test/api/fine-tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Data Preprocessing


In [ ]:

# Load dataset
# Replace with your dataset name
dataset = load_dataset('FareedKhan/1k_stories_100_genre')

# Load NER pipeline
model_name = "dslim/bert-base-NER"  # Pre-trained NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline(
    "ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

# Function to extract characters using NER


def extract_characters(example):
    story = example['story']
    entities = ner_pipeline(story)  # Run NER on the story
   # Filter out only PERSON entities and ensure uniqueness
    characters = list(
        set(entity['word'] for entity in entities if entity['entity_group'] == 'PER'))
    return {"characters": characters}


# Apply the function to add the characters column
dataset = dataset.map(extract_characters)

# Inspect the updated dataset
print(dataset)

In [ ]:
dataset['train']['characters'][:5]

In [ ]:
def format_prompt(example):
    """Create a structured and guiding prompt for story generation."""

    # Extract data from the example
    genre = example["genre"]
    # Join characters into a single string
    characters = ", ".join(example["characters"])
    story = example["story"]
    title = example["title"]

    # Create the prompt text with a guiding introduction
    prompt = (
        "Create a story based on the given genre and title. "
        f"Genre: {genre}\n"
        f"Title:\n{title}\n\n"
        "Ensure the output includes the following in order:\n"
        "1. Characters\n"
        "2. The story\n"
        "Output:\n"
        f"Characters: {characters}\n"
        f"Story: {story}\n"
    )

    return {"text": prompt}


# Apply the formatting to the dataset
formatted_dataset = dataset.map(format_prompt)

In [ ]:
print(formatted_dataset['train']['text'][:5])

In [ ]:
formatted_dataset.save_to_disk("./datasets/processed_stories")

In [ ]:
from datasets import load_from_disk

# Load the dataset from the saved directory
# Replace with the path you used earlier
dataset = load_from_disk("./datasets/processed_stories")

# Inspect the loaded dataset
print(dataset)

In [ ]:
dataset = dataset.filter(lambda example, index: index %
                         100 == 0, with_indices=True)
print(dataset)

In [3]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from datasets import load_dataset

# Load dataset with specified split
dataset = load_dataset('FareedKhan/1k_stories_100_genre', split="train")

# Shuffle and select a range
dataset = dataset.shuffle(seed=42).select(range(10))

# Load NER pipeline
model_name = "dslim/bert-base-NER"  # Pre-trained NER model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline(
    "ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=0)

# Function to extract characters using NER
def extract_characters(example):
    story = example['story']
    entities = ner_pipeline(story)  # Run NER on the story
    # Filter out only PERSON entities and ensure uniqueness
    characters = list(
        set(entity['word'] for entity in entities if entity['entity_group'] == 'PER'))
    return {"characters": characters}

# Apply the function to add the characters column
dataset = dataset.map(extract_characters)

# Function to format the prompt
def format_prompt_alpaca(example):
    """
    Create a structured and guiding prompt for story generation in Alpaca format,
    ensuring characters and story are both part of the output.
    """
    # Extract data from the example
    genre = example["genre"]
    title = example["title"]
    characters = ", ".join(example["characters"])
    story = example["story"]

    # Create the Alpaca-style prompt with instruction and input-output fields
    prompt = (
        "instruction: Generate a story based on the provided genre, title. Ensure the output includes the characters and the story."
        "input: ("
            f"Genre: {genre}\n"
            f"Title: {title}\n"
        "),"
        "output: ("
            f"Characters: {characters}\n"
            f"Story: {story}"
        ")"
    )

    return {"text": prompt}

# Apply formatting and retain only {"text": prompt}
dataset = dataset.map(format_prompt_alpaca)

# Inspect the final dataset
print(dataset)


README.md:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

1k_stories_100_genre.csv:   0%|          | 0.00/5.92M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'title', 'story', 'genre', 'characters', 'text'],
    num_rows: 10
})


In [11]:
dataset['text']

['instruction: Generate a story based on the provided genre, title. Ensure the output includes the characters and the story.input: (Genre: Mythic Fiction\nTitle: The Celestial Riddle\n),output: (Characters: Luna, ##hali, T\nStory: In a realm of myth and magic, there lived a young shepherd boy named Thalin. He tended to the celestial sheep that roamed the mystical plains.\n\n    One fateful night, while the moon bathed the plains in silver, Thalin heard a soft voice whispering a riddle. It was the voice of the moon goddess herself, Luna.\n\n    The riddle was simple yet cryptic: "What wanders the night without making a sound?"\n\n    Perplexed, Thalin pondered the riddle all night, until dawn broke and the sun kissed the horizon. He realized that the answer to the riddle was the silent shadow that follows the moon.\n\n    Filled with newfound wisdom, Thalin dedicated his life to studying the mysteries of the universe, guided by the celestial riddle.)',
 'instruction: Generate a story ba

In [5]:
from google.colab import userdata
HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN')

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Llama-3.1-8B"

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype="float16",  # Compute dtype
    bnb_4bit_use_double_quant=True,  # Apply nested quantization
    llm_int8_enable_fp32_cpu_offload=True,
)

# Load the model to train on the GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    # Leave this out for regular SFT
    quantization_config=bnb_config,
    token=HUGGINGFACE_TOKEN
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [7]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [8]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.1,  # Dropout for LoRA Layers
    r=32,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules= ['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [9]:
model.get_nb_trainable_parameters()

(83886080, 8114147328)

In [10]:
from transformers import TrainingArguments

output_dir = "./results"

# Training arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True
)

In [12]:
from trl import SFTTrainer
import os
from google.colab import output

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    max_seq_length=1024,

    # Leave this out for regular SFT
    peft_config=peft_config,
)

# Train model
trainer.train()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.2607110738754272, metrics={'train_runtime': 74.1903, 'train_samples_per_second': 0.135, 'train_steps_per_second': 0.013, 'total_flos': 301382030204928.0, 'train_loss': 1.2607110738754272, 'epoch': 0.8})

In [13]:
trainer.tokenizer.save_pretrained("Llama-3.1-8B-qlora")

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('Llama-3.1-8B-qlora/tokenizer_config.json',
 'Llama-3.1-8B-qlora/special_tokens_map.json',
 'Llama-3.1-8B-qlora/tokenizer.json')

In [14]:
merged_model = trainer.model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:336: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [15]:
merged_model.save_pretrained("Llama-3.1-8B-qlora")

In [21]:
merged_model.eval()

prompt = """Create a story based on the given genre and title.
input:
    "Genre: Romance"
    "Title: Love Story Between James and Lucy"
"output:
    "Characters: <Generate the characters based on title (if provided, if not based on your creativity)
    "Story" <Generate stories based on genre, title and characters>
Stop Here, Only 1 story is enough, generate as long as you need, and only return the story
"""

tokenized_prompt = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output
output = merged_model.generate(**tokenized_prompt, max_length=500)
print(tokenizer.decode(output[0]))
# print(tokenizer.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>Create a story based on the given genre and title.
input:
    "Genre: Romance"
    "Title: Love Story Between James and Lucy"
"output:
    "Characters: <Generate the characters based on title (if provided, if not based on your creativity)
    "Story" <Generate stories based on genre, title and characters>
Stop Here, Only 1 story is enough, generate as long as you need, and only return the story
    "The story of James and Lucy"
    "One day, James and Lucy were walking in the park. They saw a beautiful rose garden and decided to take a stroll through it. As they walked, they saw a beautiful rose bush with a single rose on it. James picked the rose and gave it to Lucy, who was touched by his gesture. They continued to walk and came upon a beautiful waterfall. They sat on a rock and watched the water fall for a while, enjoying the peace and quiet. As they were about to leave, they heard a voice call out to them. It was a man, and he was carrying a basket of apples. He of

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the model directory
model_dir = "Llama-3.1-8B-qlora"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",  # Automatically map to available GPUs
    torch_dtype="auto"  # Use FP16 or BF16 if available
)

# Set model to evaluation mode
model.eval()

# Test the loaded model with a prompt
prompt = """Create a story based on the given genre and title.
input:
    "Genre: Romance"
    "Title: Love Story Between James and Lucy"
"output:
    "Characters: <Generate the characters based on title (if provided, if not based on your creativity)
    "Story" <Generate stories based on genre, title and characters>
Stop Here, Only 1 story is enough, generate as long as you need, and only return the story
"""

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
model = model.to("cuda")
output = model.generate(**inputs, max_length=100)
print(tokenizer.decode(output[0]))


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 14.75 GiB of which 473.06 MiB is free. Process 2378 has 14.28 GiB memory in use. Of the allocated memory 13.33 GiB is allocated by PyTorch, and 836.30 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)